In [2]:
# -*- coding: utf-8 -*-
"""
@author: Etienne Kras
"""

# generic imports
import sys
import os
import geopandas as gpd
import time
import geemap
import geojson
import ee
ee.Initialize()

# specific imports
from typing import Any, Dict, List, Optional
from geojson import Polygon, Feature, FeatureCollection, dump
from shapely.geometry import Polygon
from dateutil.relativedelta import *
from google.cloud import storage
from logging import Logger, getLogger
from googleapiclient.discovery import build
from re import sub
from ctypes import ArgumentError
from functools import partial
from dateutil.parser import parse

# custom functionality import without requirement to pip install package
local_path = r"C:\Users\kras\GitHub\ee-packages-py"  # path to local GitHub clone
sys.path.append(local_path)
from eepackages.applications.bathymetry import Bathymetry
from eepackages import tiler

logger: Logger = getLogger(__name__)


C:\Users\kras\AppData\Local\Temp\ipykernel_30836\976463201.py:9: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


TODO (wishlist):

grid cell image collection
- image clipped on tile & obtain centroid
- cloud-mask with QA60?
- calculate proportion of coverge on tile
- time of acquisition in featurecollection metadata

tidal calibration
- load in GTSM info and match centriod with closest point / linear interpolation
- derive MHWS and MLWS (in m) relative to MSL
- get tidal elevation relative to MSL at time of acq. from GTSM
- convert to percentage by combining tidal elev, MHWS & MLWS
- obtain high tide & low tide offset and spread (diff. high & low tide)
- potentially get in-situ gauge data in certain cells (if applicable)

water occurrence
- calculate NDWI for each image in grid; use 0.2 or otsu to determine water / land area. Water is 1 and land is 0
- calculate water occurrence freq (perc) for all images in the collection

intertidal elevation & tidal stage
- creation of median images at set bins by means of water occurrence
- identify lower value of tidal stage in each bin & the mean tidal elevation
- create intertidal elevation from all images
- create tidal stage from all image

viewing application
- water occurrence output
- intertidal elevation output
- intertidal tide stage output
- true color image of highest tidal stage
- ture color image of lowest tidal stage
- MHWS & MLWS contours (vectors)

TODO: implement migration to NETCDF4 with WGS84 non projected lon lat relative to MSL

TODO: validate the output on in-situ data

# Project specific toggles

In [4]:
# see scheme at https://github.com/openearth/eo-bathymetry/blob/master/notebooks/rws-bathymetry/acces_api.pdf for a workflow visualization 

# project toggles
main_fol = r"p:\11209821-cmems-global-sdb" # name of the main local folder 
bucket = "cmems-sdb" # name of the Google Cloud Storage bucket to store files in the cloud
credential_file = r"p:\11209821-cmems-global-sdb\00_miscellaneous\KEYS\bathymetry-543b622ddce7.json" # Cloud Storage credential key
output_fol = "Proxy" # name of the overall project
project_name = "AOI_GER_WaddenSea" # name of the project AoI
draw_AoI = 0 # toggle 1 to draw AoI, 0 to load

# composite image toggles
mode = "intertidal" # specify mode, either "intertidal" or "subtidal"
start_date = "2021-01-01" # start date of the composites
stop_date = "2022-01-01" # end date of the composites
compo_int = 12 # composite interval [months]
compo_len = 12 # composite length [months]
scale = 30  # output resolution of the image [m]

# tiling options
zoomed_list = [9, 10, 11] # list with zoom levels to be inspected
sel_tile = 2 # idx of chosen tile level in zoomed_list (inspect the map to chose it accordingly), z9 too big for in memory computations
# note, see https://www.openearth.nl/rws-bathymetry/2019.html; Z9 is optimal size..

# load google credentials, if specified
if not credential_file == "":  
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = str(credential_file)

# Pre-processing using the API

In [18]:
# draw or load Area of Interest (AoI)

# TODO: take center of AOI input file if present, put in random coordinate and let user find a place and draw a polygon
# TODO: fix horizontal tiling error (DOS) in API (to use multiple tiles) or move to single polygon run if AoI crosses multiple tiles
Map = geemap.Map(center=(54.2, 6.7), zoom=8) # initialize map with base in Hudayriat

if draw_AoI == 1:
    print("Please draw a polygon somewhere in a water body") # identifier
if draw_AoI == 0:
    # open AoI
    print("Loading and visualizing AoI") #identifier
    #AoIee = geemap.geojson_to_ee(os.path.join(main_fol,'AOI',project_name+'.geojson'))

    with open(os.path.join(main_fol, "00_miscellaneous\AOIs", project_name + ".geojson"), 'r') as f:
        contents = geojson.loads(f.read())
    AoIee = ee.Geometry(contents["features"][0]["geometry"])

    Map.addLayer(AoIee, {}, "AoI")

Map # show map

Loading and visualizing AoI


Map(center=[54.2, 6.7], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(…

In [6]:
# (re)construct the AoI

if draw_AoI == 1:
    
    print("Constructing AoI from drawn polygon") # identifier
    
    # get AoI 
    AoIee = ee.FeatureCollection(Map.draw_features) # make featurecollection
    AoI = Polygon(AoIee.getInfo()["features"][0]["geometry"]["coordinates"][0]) # create AoI shapefile

    # export AoI
    features = []
    features.append(Feature(geometry=AoI, properties={"AoI": project_name}))
    feature_collection = FeatureCollection(features)
    with open(os.path.join(main_fol,"AOI",project_name + ".geojson"), "w") as f: # geojson
        dump(feature_collection, f)
    gdr = gpd.GeoDataFrame({"properties":{"AoI": project_name}, "geometry": AoI}, crs="EPSG:4326") #shp
    gdr.to_file(os.path.join(main_fol,"AOI",project_name+".shp"))
    bounds = ee.Geometry.Polygon([[[a,b] for a, b in zip(*AoI.exterior.coords.xy)]])
    
if draw_AoI == 0:
    print("Reconstructing AoI from loaded file")
    # get AoI
    with open(os.path.join(main_fol,"00_miscellaneous\AOIs",project_name+".geojson")) as f:
        AoIjson = geojson.load(f)
    # try: # drawn polygon in this script
    #     AoI = Polygon(AoIjson["features"][0]["geometry"]["coordinates"]) 
    # except: # drawn in QGIS / ArcGIS and written to geojson there (client file)
    #     AoI = Polygon(AoIjson["features"][0]["geometry"]["coordinates"][0])
    #bounds = ee.Geometry.Polygon([[[a,b] for a, b in zip(*AoI.exterior.coords.xy)]])
    bounds = ee.Geometry.MultiPolygon(AoIjson["features"][0]["geometry"]["coordinates"])

Reconstructing AoI from loaded file


In [23]:
# tiling the AoI
def add_tile_bounds(zoom):
    tiled = tiler.get_tiles_for_geometry(bounds, zoom)
    Map.addLayer(tiled.style(width=max(1, 10 - zoom), fillColor= "00000022"), {}, "tiles " + str(zoom))

    return(tiled)

tiles = list(map(add_tile_bounds, zoomed_list)) # add tiles for different zoom levels

In [ ]:
# Get the bounds of the AoI
print('Bounds')
bounds = ee.Geometry(AoIjson["features"][0]["geometry"])

# Get the tiles for the AoI
print('Tiles')
tiles = list(tiler.get_tiles_for_geometry(bounds, zoom) for zoom in zoomed_list)

In [ ]:
# export selected tiles (represented by zoom level) to geojsons
# note, adjust sel_tile accordingly 
# for idx, tile in enumerate(tiles[sel_tile].getInfo()["features"]):
#     tile_pol = Polygon(tile['geometry']['coordinates'][0]) # create tile polygon
#     tile_name = "z%s_x%s_y%s"%(tile["properties"]["zoom"], int(float(tile["properties"]["tx"])), int(float(tile["properties"]["ty"])))

#     features = []
#     features.append(Feature(geometry=tile_pol, properties={"name": tile_name}))
#     feature_collection = FeatureCollection(features)
#     feature_collection.crs = {"type": "name","properties": {"name": "epsg:3857"}} # default EE projection
#     with open(os.path.join(main_fol,"00_miscellaneous\AOI_polygons",tile_name + ".geojson"), "w") as f: # geojson
#         dump(feature_collection, f)

# Compute SDB using the API

In [25]:
tiled = tiles[sel_tile]
tile = ee.Feature(tiled.filterMetadata("tx", "equals", "1065.0").filterMetadata("ty", "equals", "660.0").first())

In [29]:
# Get start and stop dates
start=ee.String(start_date)
stop=ee.String(stop_date)

bounds: ee.Geometry = ee.Feature(tile).geometry().bounds(1)
sdb: Bathymetry = Bathymetry()
zoom: ee.String = ee.String(tile.get("zoom"))
tx: ee.String = ee.String(tile.get("tx"))
ty: ee.String = ee.String(tile.get("ty"))
tile_name: ee.String = ee.String("z").cat(zoom).cat("_x").cat(tx).cat("_y").cat(ty).replace("\.\d+", "", "g")
img_fullname: ee.String = ee.String(tile_name).cat("_t").cat(ee.Date(start).millis().format())
    
image: ee.Image = sdb.compute_intertidal_depth(
    bounds=bounds,
    start=start,
    stop=stop,
    scale=tiler.zoom_to_scale(ee.Number.parse(tile.get("zoom"))).multiply(5), # scale to search for clean images
    # missions=['S2', 'L8'],
    # filter: ee.Filter.dayOfYear(7*30, 9*30), # summer-only
    filter_masked=False, 
    # filterMaskedFraction = 0.5,
    # skip_scene_boundary_fix=False,
    # skip_neighborhood_search=False,
    neighborhood_search_parameters={"erosion": 0, "dilation": 0, "weight": 50},
    bounds_buffer=0,
    water_index_min=-0.05,
    water_index_max=0.15,
    # lowerCdfBoundary=45,
    # upperCdfBoundary=50,
    # cloud_frequency_threshold_data=0.15, 
    clip = True
)# .reproject(ee.Projection("EPSG:3857").atScale(90))

image = image.set(
    "fullname", img_fullname,
    "system:time_start", ee.Date(start).millis(),
    "system:time_stop", ee.Date(stop).millis(),
    "zoom", zoom,
    "tx", tx,
    "ty", ty
)


In [30]:
# Get date of image
ee_date = ee.Date(image.get("system:time_start")).format("YYYY-MM-dd HH:mm:ss")
date = ee_date.getInfo()
print(date)

2021-01-01 00:00:00


In [31]:
# Get dates of all images
ee_dates = sdb._raw_images.aggregate_array("system:time_start").map(lambda date: ee.Date(date).format("YYYY-MM-dd HH:mm:ss"))
dates = ee_dates.getInfo()
print(dates)

['2021-05-12 10:46:06', '2021-05-12 10:46:07', '2021-05-31 10:33:11', '2021-10-04 10:46:07', '2021-10-04 10:46:07', '2021-04-20 10:55:58', '2021-04-20 10:55:58', '2021-05-31 10:32:47', '2021-08-10 10:46:09', '2021-08-23 10:56:06', '2021-08-10 10:46:10', '2021-08-23 10:56:05', '2021-06-01 10:46:08', '2021-07-18 10:33:23', '2021-06-01 10:46:08', '2021-04-27 10:46:03', '2021-03-05 10:27:13', '2021-06-09 10:27:04', '2021-04-27 10:46:02', '2021-03-31 10:56:02', '2021-03-31 10:56:02', '2021-07-18 10:32:59', '2021-03-16 10:56:03', '2021-03-16 10:56:03', '2021-06-09 10:56:04', '2021-02-26 10:46:05', '2021-10-24 10:46:08', '2021-10-24 10:46:09', '2021-02-26 10:46:06', '2021-06-09 10:56:05', '2021-06-16 10:46:08', '2021-06-16 10:46:08', '2021-07-24 10:56:07', '2021-07-24 10:56:07', '2021-06-16 10:33:17', '2021-04-25 10:56:00', '2021-09-09 10:46:07', '2021-06-16 10:32:54', '2021-07-11 10:27:11', '2021-09-09 10:46:07', '2021-10-12 10:56:09', '2021-10-12 10:56:09', '2021-04-25 10:56:00', '2021-02-1

In [33]:
image.getInfo()

{'type': 'Image',
 'bands': [{'id': 'ndwi',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [1, 1],
   'origin': [7, 53],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'indvi',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [1, 1],
   'origin': [7, 53],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'mndwi',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [1, 1],
   'origin': [7, 53],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}],
 'properties': {'tx': '1065.0',
  'ty': '660.0',
  'system:time_start': 1609459200000,
  'system:footprint': {'geodesic': False,
   'type': 'Polygon',
   'coordinates': [[[7.207031251969043, 53.64463782637663],
     [7.38281250197088, 53.64463782637663],
     [7.38281250197088, 53.74871130069919],
     [7.207031251969043, 53.74871130069919],
     [7.207031251969043, 53.64463782637663]]]},
  'zoom'

In [34]:
sdb._raw_images.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'swir',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'nir',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'red',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'green',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'blue',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}],
 'properties': {'MULTIPLIER': 0.0001,
  'SOLAR_IRRADIANCE_B12': 85.25,
  'SOLAR_IRRADIANCE_B10': 367.15,
  'SENSOR_QUALITY': 'PASSED',
  'SOLAR_IRRADIANCE_B11': 245.59,
  'BANDS_FROM': ['B11', 'B8', 'B4', 'B3', 'B2'],
  'GENERATION_TIME': 1620824443000,
  'SUN_AZ

In [36]:
# Plot all raw images
Map = geemap.Map()
Map.centerObject(bounds, 9)
Map.addLayer(bounds, {}, "AoI")
# from tqdm import tqdm
# for idx in tqdm(range(sdb._raw_images.size().getInfo())):
#     image_ = ee.Image(sdb._raw_images.toList(sdb._raw_images.size()).get(idx))
#     Map.addLayer(image_, {"bands": ["red", "green", "blue"], "min": 0, "max": 0.3}, f"image_{idx} (date: {dates[idx]})")
#     if idx > 1:
#         break
Map

Map(center=[53.696663843187544, 7.294921876971232], controls=(WidgetControl(options=['position', 'transparent_…

In [38]:
Map.addLayer(sdb._raw_images.first().clip(bounds), {"bands": ["red", "green", "blue"], "min": 0, "max": 0.3}, f"image_{idx} (date: {dates[idx]})")